- 다음모임때 준비할 것 
  . 본인 회사 주식, 2018년부터 지금까지 데이터 확보하기 (교재는 2021년까지임.)
    : 증권회사 또는 네이버 등 본인이 편한 방법으로
  . 잘 얻어졌다면 업데이트 된 데이터로 프로그램 돌려보기 (옵션임)


date, open, high, low, close, volume, per, pbr, roe,
open_lastclose_ratio, high_close_ratio, low_close_ratio, diffratio, volume_lastvolume_ratio,

close_ma5_ratio, volume_ma5_ratio, close_ma10_ratio, volume_ma10_ratio, close_ma20_ratio, volume_ma20_ratio,
close_ma60_ratio, volume_ma60_ratio, close_ma120_ratio, volume_ma120_ratio,

ind,ind_diff,ind_ma5,ind_ma10,ind_ma20,ind_ma60,ind_ma120,
inst,inst_diff,inst_ma5,inst_ma10,inst_ma20,inst_ma60,inst_ma120,
foreign,foreign_diff,foreign_ma5,foreign_ma10,foreign_ma20,foreign_ma60,foreign_ma120

In [19]:
import FinanceDataReader as fdr
from pykrx import stock
# import pandas_datareader as pdr
import pandas_datareader.data as web
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [20]:
# 데이터 수집 및 머지

# LS ELECTRIC 010120 / POSCODX 022100
# 120일 이동평균 계산 위해 2017년도부터 데이터 취득
df1 = fdr.DataReader('010120', '2017-01-01', '2023-12-31')
df2 = stock.get_market_fundamental("20170101", "20231231", "010120")
df3 = stock.get_market_trading_value_by_date("20170101", "20231231", "010120")

In [21]:
df_temp = df1.merge(df2, left_index=True, right_index=True, how='inner')
df = df_temp.merge(df3, left_index=True, right_index=True, how='inner')

df.head()

,Open,High,Low,Close,Volume,Change,BPS,PER,PBR,EPS,DIV,DPS,기관합계,기타법인,개인,외국인합계,전체
Date,,,,,,,,,,,,,,,,,
2017-01-02,39950,40050,39700,40050,20231,0.007547,33775,16.71,1.19,2397,2.50,1000,91850650,0,-133308100,41457450,0
2017-01-03,40000,40000,39150,39150,121742,-0.022472,33775,16.33,1.16,2397,2.55,1000,-2092469700,31344500,2738869500,-677744300,0
2017-01-04,39050,39250,38900,38950,57463,-0.005109,33775,16.25,1.15,2397,2.57,1000,-1085358900,37972650,806594450,240791800,0
2017-01-05,38850,39350,38700,39200,71105,0.006418,33775,16.35,1.16,2397,2.55,1000,-1048913450,27691000,1156097250,-134874800,0
2017-01-06,39200,39400,38950,39250,45566,0.001276,33775,16.37,1.16,2397,2.55,1000,-517476800,-985750,581898000,-63435450,0


In [22]:
# roe 구하기
df['ROE'] = df['PBR'] / df['PER']

# 컬럼명 정리
df.rename(columns={'개인': 'ind'}, inplace=True)
df.rename(columns={'기관합계': 'inst'}, inplace=True)
df.rename(columns={'외국인합계': 'foreign'}, inplace=True)

In [23]:
# index를 date 컬럼으로 바꾸고 "yyyymmdd"(int) 형태로 변환
df = df.reset_index()
df['Date'] = df['Date'].dt.strftime("%Y%m%d")
df = df.astype({"Date": int})
df.head()

,Date,Open,High,Low,Close,Volume,Change,BPS,PER,PBR,EPS,DIV,DPS,inst,기타법인,ind,foreign,전체,ROE
0,20170102,39950,40050,39700,40050,20231,0.007547,33775,16.71,1.19,2397,2.50,1000,91850650,0,-133308100,41457450,0,0.071215
1,20170103,40000,40000,39150,39150,121742,-0.022472,33775,16.33,1.16,2397,2.55,1000,-2092469700,31344500,2738869500,-677744300,0,0.071035
2,20170104,39050,39250,38900,38950,57463,-0.005109,33775,16.25,1.15,2397,2.57,1000,-1085358900,37972650,806594450,240791800,0,0.070769
3,20170105,38850,39350,38700,39200,71105,0.006418,33775,16.35,1.16,2397,2.55,1000,-1048913450,27691000,1156097250,-134874800,0,0.070948
4,20170106,39200,39400,38950,39250,45566,0.001276,33775,16.37,1.16,2397,2.55,1000,-517476800,-985750,581898000,-63435450,0,0.070861


In [24]:
# 컬럼 이름을 소문자로 변환
df.columns = df.columns.str.lower()

# 컬럼 삭제
df = df.drop('기타법인', axis=1)
df = df.drop('전체', axis=1)

df.head()

,date,open,high,low,close,volume,change,bps,per,pbr,eps,div,dps,inst,ind,foreign,roe
0,20170102,39950,40050,39700,40050,20231,0.007547,33775,16.71,1.19,2397,2.50,1000,91850650,-133308100,41457450,0.071215
1,20170103,40000,40000,39150,39150,121742,-0.022472,33775,16.33,1.16,2397,2.55,1000,-2092469700,2738869500,-677744300,0.071035
2,20170104,39050,39250,38900,38950,57463,-0.005109,33775,16.25,1.15,2397,2.57,1000,-1085358900,806594450,240791800,0.070769
3,20170105,38850,39350,38700,39200,71105,0.006418,33775,16.35,1.16,2397,2.55,1000,-1048913450,1156097250,-134874800,0.070948
4,20170106,39200,39400,38950,39250,45566,0.001276,33775,16.37,1.16,2397,2.55,1000,-517476800,581898000,-63435450,0.070861


In [25]:
# 모든 열의 데이터 타입을 숫자형으로 변환
# df = df.apply(pd.to_numeric, errors='coerce')
df.dtypes

date         int64
open         int64
high         int64
low          int64
close        int64
volume       int64
change     float64
bps          int32
per        float64
pbr        float32
eps          int32
div        float32
dps          int32
inst         int64
ind          int64
foreign      int64
roe        float64
dtype: object

In [26]:
# 비율 파생변수 last 접두사는 전일을 의미함
df['open_lastclose_ratio'] = df['open'] / df['close'].shift(1)
df['high_close_ratio'] = df['high'] / df['close']
df['low_close_ratio'] = df['low'] / df['close']
df['diffratio'] = df['open'] / df['close']
df['volume_lastvolume_ratio'] = df['volume'] / df['volume'].shift(1)


In [27]:
# 종가 이동평균으로 나눈 현재 종가 비율
df['close_ma5_ratio'] = df['close'] / df['close'].rolling(window=5).mean()
df['close_ma10_ratio'] = df['close'] / df['close'].rolling(window=10).mean()
df['close_ma20_ratio'] = df['close'] / df['close'].rolling(window=20).mean()
df['close_ma60_ratio'] = df['close'] / df['close'].rolling(window=60).mean()
df['close_ma120_ratio'] = df['close'] / df['close'].rolling(window=120).mean()

# 거래량 이동평균으로 나눈 현재 거래량 비율
df['volume_ma5_ratio'] = df['volume'] / df['volume'].rolling(window=5).mean()
df['volume_ma10_ratio'] = df['volume'] / df['volume'].rolling(window=10).mean()
df['volume_ma20_ratio'] = df['volume'] / df['volume'].rolling(window=20).mean()
df['volume_ma60_ratio'] = df['volume'] / df['volume'].rolling(window=60).mean()
df['volume_ma120_ratio'] = df['volume'] / df['volume'].rolling(window=120).mean()
df.head()

,date,open,high,low,close,volume,change,bps,per,pbr,...,close_ma5_ratio,close_ma10_ratio,close_ma20_ratio,close_ma60_ratio,close_ma120_ratio,volume_ma5_ratio,volume_ma10_ratio,volume_ma20_ratio,volume_ma60_ratio,volume_ma120_ratio
0,20170102,39950,40050,39700,40050,20231,0.007547,33775,16.71,1.19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20170103,40000,40000,39150,39150,121742,-0.022472,33775,16.33,1.16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20170104,39050,39250,38900,38950,57463,-0.005109,33775,16.25,1.15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20170105,38850,39350,38700,39200,71105,0.006418,33775,16.35,1.16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20170106,39200,39400,38950,39250,45566,0.001276,33775,16.37,1.16,...,0.99822,NaN,NaN,NaN,NaN,0.720737,NaN,NaN,NaN,NaN


In [28]:
# ind,ind_diff,ind_ma5,ind_ma10,ind_ma20,ind_ma60,ind_ma120,
# inst,inst_diff,inst_ma5,inst_ma10,inst_ma20,inst_ma60,inst_ma120,
# foreign,foreign_diff,foreign_ma5,foreign_ma10,foreign_ma20,foreign_ma60,foreign_ma120

# 개인 거래량 이동평균
df['ind_diff'] = df['ind'] - df['ind'].shift(1)
df['ind_ma5'] = df['ind'].rolling(window=5).mean()
df['ind_ma10'] = df['ind'].rolling(window=10).mean()
df['ind_ma20'] = df['ind'].rolling(window=20).mean()
df['ind_ma60'] = df['ind'].rolling(window=60).mean()
df['ind_ma120'] = df['ind'].rolling(window=120).mean()

# 기관 거래량 이동평균
df['inst_diff'] = df['inst'] - df['inst'].shift(1)
df['inst_ma5'] = df['inst'].rolling(window=5).mean()
df['inst_ma10'] = df['inst'].rolling(window=10).mean()
df['inst_ma20'] = df['inst'].rolling(window=20).mean()
df['inst_ma60'] = df['inst'].rolling(window=60).mean()
df['inst_ma120'] = df['inst'].rolling(window=120).mean()

# 외국인 거래량 이동평균
df['foreign_diff'] = df['foreign'] - df['foreign'].shift(1)
df['foreign_ma5'] = df['inst'].rolling(window=5).mean()
df['foreign_ma10'] = df['inst'].rolling(window=10).mean()
df['foreign_ma20'] = df['inst'].rolling(window=20).mean()
df['foreign_ma60'] = df['inst'].rolling(window=60).mean()
df['foreign_ma120'] = df['inst'].rolling(window=120).mean()
df.head()

,date,open,high,low,close,volume,change,bps,per,pbr,...,inst_ma10,inst_ma20,inst_ma60,inst_ma120,foreign_diff,foreign_ma5,foreign_ma10,foreign_ma20,foreign_ma60,foreign_ma120
0,20170102,39950,40050,39700,40050,20231,0.007547,33775,16.71,1.19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20170103,40000,40000,39150,39150,121742,-0.022472,33775,16.33,1.16,...,NaN,NaN,NaN,NaN,-719201750.0,NaN,NaN,NaN,NaN,NaN
2,20170104,39050,39250,38900,38950,57463,-0.005109,33775,16.25,1.15,...,NaN,NaN,NaN,NaN,918536100.0,NaN,NaN,NaN,NaN,NaN
3,20170105,38850,39350,38700,39200,71105,0.006418,33775,16.35,1.16,...,NaN,NaN,NaN,NaN,-375666600.0,NaN,NaN,NaN,NaN,NaN
4,20170106,39200,39400,38950,39250,45566,0.001276,33775,16.37,1.16,...,NaN,NaN,NaN,NaN,71439350.0,-930473640.0,NaN,NaN,NaN,NaN


In [29]:
# 2018년 이전 행 제거
new_df = df[df['date'] > 20180000].reset_index(drop=True)
new_df.head()

,date,open,high,low,close,volume,change,bps,per,pbr,...,inst_ma10,inst_ma20,inst_ma60,inst_ma120,foreign_diff,foreign_ma5,foreign_ma10,foreign_ma20,foreign_ma60,foreign_ma120
0,20180102,65100,67000,65100,66900,71733,0.027650,35486,24.31,1.89,...,1.740235e+09,8.417265e+08,1.360034e+08,-1.670437e+08,-636450800.0,1.478592e+09,1.740235e+09,8.417265e+08,1.360034e+08,-1.670437e+08
1,20180103,65700,66700,65100,65600,42991,-0.019432,35486,23.84,1.85,...,1.287584e+09,8.333931e+08,1.396809e+08,-1.552751e+08,154087700.0,9.762774e+08,1.287584e+09,8.333931e+08,1.396809e+08,-1.552751e+08
2,20180104,65900,65900,64700,65100,40764,-0.007622,35486,23.66,1.83,...,1.007821e+09,8.896487e+08,1.556489e+08,-1.331585e+08,137633000.0,2.569546e+08,1.007821e+09,8.896487e+08,1.556489e+08,-1.331585e+08
3,20180105,65800,67500,64900,67100,85647,0.030722,35486,24.38,1.89,...,1.278790e+09,1.053403e+09,2.227351e+08,-1.172455e+08,67614600.0,7.907526e+08,1.278790e+09,1.053403e+09,2.227351e+08,-1.172455e+08
4,20180108,67500,67800,66300,67300,66679,0.002981,35486,24.45,1.90,...,1.064499e+09,1.076983e+09,2.752806e+08,-1.085070e+08,-204646300.0,9.948044e+08,1.064499e+09,1.076983e+09,2.752806e+08,-1.085070e+08


In [30]:
ref_df = pd.read_csv("../RL_Trader/data/v3/005930_삼성전자.csv")
ref_cols = list(ref_df.columns)

In [31]:
base_cols = list(df.columns)
len(base_cols)

50

In [32]:
len(ref_cols)

45

In [33]:
unseen_cols = [col for col in base_cols if col not in ref_cols]
unseen_cols

['change', 'bps', 'eps', 'div', 'dps']

In [34]:
new_df = new_df[ref_cols]
new_df.head()

,date,open,high,low,close,volume,per,pbr,roe,open_lastclose_ratio,...,inst_ma20,inst_ma60,inst_ma120,foreign,foreign_diff,foreign_ma5,foreign_ma10,foreign_ma20,foreign_ma60,foreign_ma120
0,20180102,65100,67000,65100,66900,71733,24.31,1.89,0.077746,1.000000,...,8.417265e+08,1.360034e+08,-1.670437e+08,-347614400,-636450800.0,1.478592e+09,1.740235e+09,8.417265e+08,1.360034e+08,-1.670437e+08
1,20180103,65700,66700,65100,65600,42991,23.84,1.85,0.077601,0.982063,...,8.333931e+08,1.396809e+08,-1.552751e+08,-193526700,154087700.0,9.762774e+08,1.287584e+09,8.333931e+08,1.396809e+08,-1.552751e+08
2,20180104,65900,65900,64700,65100,40764,23.66,1.83,0.077346,1.004573,...,8.896487e+08,1.556489e+08,-1.331585e+08,-55893700,137633000.0,2.569546e+08,1.007821e+09,8.896487e+08,1.556489e+08,-1.331585e+08
3,20180105,65800,67500,64900,67100,85647,24.38,1.89,0.077523,1.010753,...,1.053403e+09,2.227351e+08,-1.172455e+08,11720900,67614600.0,7.907526e+08,1.278790e+09,1.053403e+09,2.227351e+08,-1.172455e+08
4,20180108,67500,67800,66300,67300,66679,24.45,1.90,0.077710,1.005961,...,1.076983e+09,2.752806e+08,-1.085070e+08,-192925400,-204646300.0,9.948044e+08,1.064499e+09,1.076983e+09,2.752806e+08,-1.085070e+08


In [35]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458 entries, 0 to 1457
Data columns (total 45 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   date                     1458 non-null   int64  
 1   open                     1458 non-null   int64  
 2   high                     1458 non-null   int64  
 3   low                      1458 non-null   int64  
 4   close                    1458 non-null   int64  
 5   volume                   1458 non-null   int64  
 6   per                      1458 non-null   float64
 7   pbr                      1458 non-null   float32
 8   roe                      1458 non-null   float64
 9   open_lastclose_ratio     1458 non-null   float64
 10  high_close_ratio         1458 non-null   float64
 11  low_close_ratio          1458 non-null   float64
 12  diffratio                1458 non-null   float64
 13  volume_lastvolume_ratio  1458 non-null   float64
 14  close_ma5_ratio         

In [37]:
# 데이터프레임을 CSV 파일로 저장
new_df.to_csv('../RL_Trader/data/v3/010120_lselecric.csv', index=False)